<a href="https://colab.research.google.com/github/farhanwadia/EEGClassification/blob/Farhan/EEG_Classification_Model_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EEG Classification Model Testing

## 1. Clone the GitHub Repository

In [1]:
!git clone https://github.com/farhanwadia/EEGClassification
!cd EEGClassification && git checkout Farhan

#Change the working directory to the EEGClassification folder 
%cd EEGClassification
!pwd

Cloning into 'EEGClassification'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 10615 (delta 27), reused 24 (delta 8), pack-reused 10538
Receiving objects: 100% (10615/10615), 4.00 GiB | 24.96 MiB/s, done.
Resolving deltas: 100% (537/537), done.
Checking out files: 100% (10468/10468), done.
Branch 'Farhan' set up to track remote branch 'Farhan' from 'origin'.
Switched to a new branch 'Farhan'
/content/EEGClassification
/content/EEGClassification


### Imports

In [2]:
#!pip install mne

import numpy as np
import os
#import mne
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import pickle

import csv
import json
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim

## 2. Split data into training, validation, and test sets

In [3]:
def loadCSV(filename):
    # Takes in a string of the csv file name where the EEG data is (# of measurements by 4 channels)
    # Returns the data as an np array, transposed, with all values /10**6
    data = np.loadtxt(filename, delimiter=',')
    data = data.T / 10**6
    return data

def jsonToNp(pathToJSON):
    with open(pathToJSON, "r") as f:
        data = np.array(json.load(f))
    return data 

def getFiles(parentPath):
    # Returns all files in the folder and its subfolders as a list
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(parentPath):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
    return listOfFiles

def createDataset(openFiles, closedFiles):
    n = len(openFiles) + len(closedFiles)
    w = loadCSV(openFiles[0]).shape[0]
    h = loadCSV(openFiles[0]).shape[1]

    X = np.zeros((n, w, h))
    y = np.zeros((n))
    filenames = []

    # Put files into np arrays. 
    #Assumes all data has already been cut to correct lengths, with no inconsistincies in the numper of data points between files
    i = 0
    for lst in [openFiles, closedFiles]:
        for file in lst:
            data = loadCSV(file)
            X[i] = data
            if lst == openFiles:
                y[i] = 1            #Label 1: Open
            else:
                y[i] = 0            #Label 0: Closed

            name = file.split(r"\\")[-1].split(".")[0]
            filenames.append(name)
            i = i+1
    return X, y, filenames

def createDatasetFromJSON(openFiles, closedFiles):
    n = len(openFiles) + len(closedFiles)
    c = jsonToNp(openFiles[0]).shape[0]
    h = jsonToNp(openFiles[0]).shape[1]
    w = jsonToNp(openFiles[0]).shape[2]

    X = np.zeros((n, c, h, w))
    y = np.zeros((n))
    filenames = []
    
    # Put files into np arrays.
    i = 0
    for lst in [openFiles, closedFiles]:
        for file in lst:
            data = jsonToNp(file)
            X[i] = data
            if lst == openFiles:
                y[i] = 1            #Label 1: Open
            else:
                y[i] = 0            #Label 0: Closed

            name = file.split(r"\\")[-1].split(".")[0]
            filenames.append(name)
            i = i+1
    return X, y, filenames


def shuffleDataset(X, y, filenames):
    indices = np.arange(X.shape[0])
    seed = 21 
    np.random.seed(seed)
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    filenames = np.array(filenames)
    filenames = filenames[indices]
    return X, y, filenames

def splitDataset(X, y, filenames, percent_train):
    n = X.shape[0]
    X_train = X[:int(percent_train*n)]
    X_val = X[int(percent_train*n):]
    y_train = y[:int(percent_train*n)]
    y_val = y[int(percent_train*n):]
    filenames_train = filenames[:int(percent_train*n)]
    filenames_val = filenames[int(percent_train*n):]
    return X_train, y_train, filenames_train, X_val, y_val, filenames_val

### Time Series

In [4]:
openPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//OPEN//"
closedPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//CLOSE//"
openPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//OPEN//"
closedPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//CLOSE//"
openFiles = getFiles(openPath) + getFiles(openPath2)
closedFiles = getFiles(closedPath) + getFiles(closedPath2) 

X, y, filenames = createDataset(openFiles, closedFiles)
X, y, filenames = shuffleDataset(X, y, filenames)
X_train, y_train, filenames_train, X_other, y_other, filenames_other = splitDataset(X, y, filenames, percent_train=0.7)
X_val, y_val, filenames_val, X_test, y_test, filenames_test = splitDataset(X_other, y_other, filenames_other, percent_train=0.5) 

print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)
print("X_val Shape: ", X_val.shape)
print("y_val Shape: ", y_val.shape)

X_train Shape:  (350, 4, 1280)
y_train Shape:  (350,)
X_val Shape:  (75, 4, 1280)
y_val Shape:  (75,)


Print the filenames for the datasets (can verify with training notebook that all sets are the same and that test data wasn't used for training)

In [5]:
print(filenames_train)

['/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T5 Open (valve)_(1) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T38 Close (valve) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T9 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//AZ_T38 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW_T12 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ2_T2 Close (valve) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T39 Open (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW3_T16 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T35 Open (valve)_(2) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//FW_T75 Open (valve) - Filtered'
 '/content/EEGClassi

In [6]:
print(filenames_val)

['/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T15 Open (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T24 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T32 Close (valve) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//SD_T15 Close (valve) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//FW_T16 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//CJ_T17 Open (valve)_(2) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T34 Close (valve)_(1) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//AZ_T34 Open (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T3 Open (valve) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T30 Open (valve) - Filtered'
 '/content/EEGClassifi

In [7]:
print(filenames_test)

['/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//CJ_T36 Close (valve) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//OPEN//SD_T17 Open (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//JP_T15 Open (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//FW3_T39 Open (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T30 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//AZ_T35 Open (valve) - Filtered'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//CLOSE//CJ_T75 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//FW2_T20 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//CLOSE//JP_T16 Close (valve) - Filtered'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//OPEN//SD_T21 Open (valve) - Filtered'
 '/content/EEGClassificatio

### Time-Frequency (Wavelets Transformed)

In [8]:
#openPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//Spectograms//OPEN//"
#closedPath = os.getcwd() + r"//Nov 2020 Filtered Data//5s//Spectograms//CLOSE//"
#openPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//Spectograms//OPEN//"
#closedPath2 = os.getcwd() + r"//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//"
#openFiles = getFiles(openPath) + getFiles(openPath2)
#closedFiles = getFiles(closedPath) + getFiles(closedPath2) 

#X, y, filenames = createDatasetFromJSON(openFiles, closedFiles)
#X, y, filenames = shuffleDataset(X, y, filenames)
#X_train_wv, y_train_wv, filenames_train_wv, X_other, y_other, filenames_other = splitDataset(X, y, filenames, percent_train=0.7)
#X_val_wv, y_val_wv, filenames_val_wv, X_test_wv, y_test_wv, filenames_test_wv = splitDataset(X_other, y_other, filenames_other, percent_train=0.5) 

#print("X_train_wv Shape: ", X_train_wv.shape)
#print("y_train_wv Shape: ", y_train_wv.shape)
#print("X_val_wv Shape: ", X_val_wv.shape)
#print("y_val_wv Shape: ", y_val_wv.shape)

X_train_wv Shape:  (350, 4, 60, 1280)
y_train_wv Shape:  (350,)
X_val_wv Shape:  (75, 4, 60, 1280)
y_val_wv Shape:  (75,)


Print the filenames for the datasets (can verify with training notebook that all sets are the same and that test data wasn't used for training)

In [9]:
#print(filenames_train_wv)

['/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//CJ_T33 Open (valve)_(1) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//JP_T4 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//Spectograms//CLOSE//AZ_T59 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//FW2_T20 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//CJ_T30 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//FW_T12 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//Spectograms//OPEN//SD_T32 Open (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//SD_T16 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//FW2_T31 Open (valve

In [10]:
#print(filenames_val_wv)

['/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//FW_T27 Open (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//CJ_T32 Close (valve)_(2) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//CJ_T40 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//Spectograms//CLOSE//AZ_T9 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//Spectograms//CLOSE//SD_T52 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//CJ_T39 Open (valve)_(1) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//AZ_T30 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//Spectograms//OPEN//FW_T22 Open (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//FW2_T33 Open (valv

In [11]:
#print(filenames_test_wv)

['/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//FW_T10 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//Spectograms//OPEN//FW_T6 Open (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//FW3_T3 Open (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//CJ_T19 Open (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//AZ_T14 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//OPEN//CJ_T39 Open (valve) - Spec Coefs'
 '/content/EEGClassification//Nov 2020 Filtered Data//5s//Spectograms//CLOSE//AZ_T23 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//FW2_T40 Close (valve) - Spec Coefs'
 '/content/EEGClassification//Feb 2021 Filtered Data//5s//Spectograms//CLOSE//CJ_T26 Close (valve)_(2) -

## 3. Testing

### Test the baseline SVM

In [12]:
def get_accuracy_svm(model, X, y):
    predictions = model.predict(X)
    num_correct = np.sum((predictions - y) == 0)
    n = X.shape[0]
    accuracy = num_correct/n
    print("The number of correct predictions is: ", num_correct)
    print("The number of files is: ", n)
    print("The accuracy is: ", accuracy)

In [13]:
#Flatten channels to single vector for SVM
X_train_svm = X_train.reshape(X_train.shape[0], 4*(256*5))
X_val_svm = X_val.reshape(X_val.shape[0], 4*(256*5))
X_test_svm = X_test.reshape(X_test.shape[0], 4*(256*5))

In [14]:
filename = 'Trained Model Weights/svm_kernel-sigmoid_C-100_5s.sav'
model = pickle.load(open(filename, 'rb'))

In [15]:
print("Training Accuracy")
get_accuracy_svm(model, X_train_svm, y_train)
print("\nValidation Accuracy")
get_accuracy_svm(model, X_val_svm, y_val)

Training Accuracy
The number of correct predictions is:  240
The number of files is:  350
The accuracy is:  0.6857142857142857

Validation Accuracy
The number of correct predictions is:  58
The number of files is:  75
The accuracy is:  0.7733333333333333


In [16]:
print("Test Accuracy")
get_accuracy_svm(model, X_test_svm, y_test)

Test Accuracy
The number of correct predictions is:  49
The number of files is:  75
The accuracy is:  0.6533333333333333


### Test the ANN models

In [17]:
def createTensorDataset(data, labels):
    # Returns a tensor dataset object given np arrays of the data and labels
    data_tensor = torch.tensor(data)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(data_tensor, labels_tensor)
    return dataset

def get_accuracy(model, data_loader, train=False, use_cuda=True):
    correct = 0
    total = 0
    for file, labels in iter(data_loader):
        file = file.float()
        labels = labels.long()
        if use_cuda and torch.cuda.is_available():
            file = file.cuda()
            labels = labels.cuda()
        output = model(file)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += file.shape[0]
    return correct / total

def print_model_predictions(model, data_loader, use_cuda=True):
    #Use a batch size of 1 for the data_loader passed in
    num_correct = 0
    total = 0
    for file, labels in iter(data_loader):
        file = file.float()
        labels = labels.long()
        if use_cuda and torch.cuda.is_available():
            file = file.cuda()
            labels = labels.cuda()
        output = model(file)
        pred = output.max(1, keepdim=True)[1]
        if pred.item() == labels.item():
            status = "Prediction Correct"
            num_correct += 1
        else:
            status = "Prediction Incorrect"
        total += 1
        print("Prediction: ", pred.item(), " | Label: ", labels.item(), " | Status: ", status)
    print("Files classified correctly: ", num_correct)
    print("Total files: ", total)
    print("Accuracy: ", num_correct/total)

In [18]:
class ANN_TS_2L(nn.Module):
    def __init__(self):
        super(ANN_TS_2L, self).__init__()
        self.name = "ANN_TS_2L"
        self.layer1 = nn.Linear((256*5)*4, 200)
        self.layer2 = nn.Linear(200, 50)
        self.layer3 = nn.Linear(50, 2)
    def forward(self, img):
        flattened = img.view(-1, (256*5)*4)
        activation1 = F.relu(self.layer1(flattened))
        activation2 = F.relu(self.layer2(activation1))
        output = self.layer3(activation2)
        return output

class ANN_TS_3L(nn.Module):
    def __init__(self):
        super(ANN_TS_3L, self).__init__()
        self.name = "ANN_TS_3L"
        self.layer1 = nn.Linear((256*5)*4, 500)
        self.layer2 = nn.Linear(500, 200)
        self.layer3 = nn.Linear(200, 50)
        self.layer4 = nn.Linear(50, 2)
    def forward(self, img):
        flattened = img.view(-1, (256*5)*4)
        activation1 = F.relu(self.layer1(flattened))
        activation2 = F.relu(self.layer2(activation1))
        activation3 = F.relu(self.layer3(activation2))
        output = self.layer4(activation3)
        return output

#### 2 Layer ANN on 5s Recordings

##### Trained with batch_size=50

In [20]:
pth = 'Trained Model Weights/ANN_TS_2L Weights 2021-03-14 20-28/model_ANN_TS_2L_bs50_lr0-01_epoch67_iteration475_val-acc62-6667.pth'
model_2L = ANN_TS_2L()
model_2L.load_state_dict(torch.load(pth))
model_2L.eval()

ANN_TS_2L(
  (layer1): Linear(in_features=5120, out_features=200, bias=True)
  (layer2): Linear(in_features=200, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=2, bias=True)
)

In [21]:
batch_size = 1

train_dataset = createTensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

val_dataset = createTensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

test_dataset = createTensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [22]:
print("Training Set Accuracy")
get_accuracy(model_2L, train_loader)

Training Set Accuracy


0.8314285714285714

In [23]:
print("Validation Set Accuracy")
get_accuracy(model_2L, val_loader)

Validation Set Accuracy


0.8133333333333334

In [24]:
print("Test Set Accuracy")
get_accuracy(model_2L, test_loader)

Test Set Accuracy


0.9066666666666666

In [25]:
print_model_predictions(model_2L, test_loader)

Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction

##### Trained with batch_size=75

In [26]:
pth = 'Trained Model Weights/ANN_TS_2L Weights 2021-03-14 20-46/model_ANN_TS_2L_bs75_lr0-01_epoch93_iteration466_val-acc62-6667.pth'
model_2L_bs75 = ANN_TS_2L()
model_2L_bs75.load_state_dict(torch.load(pth))
model_2L_bs75.eval()

ANN_TS_2L(
  (layer1): Linear(in_features=5120, out_features=200, bias=True)
  (layer2): Linear(in_features=200, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=2, bias=True)
)

In [27]:
print("Training Set Accuracy")
get_accuracy(model_2L_bs75, train_loader)

Training Set Accuracy


0.8314285714285714

In [28]:
print("Validation Set Accuracy")
get_accuracy(model_2L_bs75, val_loader)

Validation Set Accuracy


0.7866666666666666

In [29]:
print("Testing Set Accuracy")
get_accuracy(model_2L_bs75, test_loader)

Testing Set Accuracy


0.84

In [30]:
print_model_predictions(model_2L_bs75, test_loader)

Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction

#### 3 Layer ANN on 5s Recordings

In [31]:
pth = 'Trained Model Weights/ANN_TS_3L Weights 2021-03-14 20-38/model_ANN_TS_3L_bs50_lr0-001_epoch72_iteration509_val-acc65-3333.pth'
model_3L = ANN_TS_3L()
model_3L.load_state_dict(torch.load(pth))
model_3L.eval()

ANN_TS_3L(
  (layer1): Linear(in_features=5120, out_features=500, bias=True)
  (layer2): Linear(in_features=500, out_features=200, bias=True)
  (layer3): Linear(in_features=200, out_features=50, bias=True)
  (layer4): Linear(in_features=50, out_features=2, bias=True)
)

In [32]:
print("Training Set Accuracy")
get_accuracy(model_3L, train_loader)

Training Set Accuracy


0.8542857142857143

In [33]:
print("Validation Set Accuracy")
get_accuracy(model_3L, val_loader)

Validation Set Accuracy


0.8266666666666667

In [34]:
print("Test Set Accuracy")
get_accuracy(model_3L, test_loader)

Test Set Accuracy


0.8666666666666667

In [35]:
print_model_predictions(model_3L, test_loader)

Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction

### Test the CNN model

In [36]:
# Upload the file manually since it is too large to store on GitHub
#from google.colab import files
#uploaded = files.upload()

Saving model_CNN_WV_bs20_lr0-00025_epoch3_iteration58_val-acc64-0.pth to model_CNN_WV_bs20_lr0-00025_epoch3_iteration58_val-acc64-0.pth


In [37]:
def downsampleTensorHW(dataset, factor):
    # Given a tensor dataset of data, labels, takes data of shape [N, C, H, W]
    # Returns the dataset, but with data of shape [N, C, H/factor, W/factor]
    data = dataset.tensors[0]
    labels = dataset.tensors[1]
    data_new = F.interpolate(data, size=(int(data.shape[-2] /factor), int(data.shape[-1] /factor)), mode='bicubic', align_corners=False)
    return TensorDataset(data_new, labels)

In [38]:
class CNN_WV(nn.Module):
    def __init__(self):
        super(CNN_WV, self).__init__()
        self.name = "CNN_WV"
        
        self.conv1 = nn.Conv2d(4, 8, 4) #in_channels, out_channels (# of kernels to try), kernel_size. Size in is 4x30x640
        self.pool = nn.MaxPool2d(2, 2) #kernel_size (for the pool), stride. Size in should be 8x26x636 
        self.conv2 = nn.Conv2d(8, 16, 4) #size in should be 8x13x318
        self.pool = nn.MaxPool2d(2, 2) # size in should be 16x9x314
        
        self.fc1 = nn.Linear(16*5*157, 2500) #size in should be 16x5x157
        self.fc2 = nn.Linear(2500, 500)
        self.fc3 = nn.Linear(500, 50)
        self.fc4 = nn.Linear(50, 2)
 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) #flatten the input

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x)) 
        x = self.fc4(x)
        x = x.squeeze(1)
        return x

In [39]:
#pth = 'model_CNN_WV_bs20_lr0-00025_epoch3_iteration58_val-acc64-0.pth'
#model_CNN = CNN_WV()
#model_CNN.load_state_dict(torch.load(pth))
#model_CNN.eval()

CNN_WV(
  (conv1): Conv2d(4, 8, kernel_size=(4, 4), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=12560, out_features=2500, bias=True)
  (fc2): Linear(in_features=2500, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=2, bias=True)
)

In [40]:
#batch_size = 1

#train_dataset = createTensorDataset(X_train_wv, y_train_wv)
#train_dataset = downsampleTensorHW(train_dataset, 2)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

#val_dataset = createTensorDataset(X_val_wv, y_val_wv)
#val_dataset = downsampleTensorHW(val_dataset, 2)
#val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

#test_dataset = createTensorDataset(X_test_wv, y_test_wv)
#test_dataset = downsampleTensorHW(test_dataset, 2)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [41]:
#print("Training Set Accuracy")
#get_accuracy(model_CNN, train_loader)

Training Set Accuracy


0.6628571428571428

In [42]:
#print("Validation Set Accuracy")
#get_accuracy(model_CNN, val_loader)

Validation Set Accuracy


0.56

In [43]:
#print("Test Set Accuracy")
#get_accuracy(model_CNN, test_loader)

Test Set Accuracy


0.6

In [44]:
#print_model_predictions(model_CNN, test_loader)

Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect
Prediction:  1  | Label:  1  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect
Prediction:  1  | Label:  0  | Status:  Prediction Incorrect
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  0  | Status:  Prediction Correct
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect
Prediction:  0  | Label:  1  | Status:  Prediction Incorrect
Prediction:  1  | Label:  0  | Status:  

## 4. Discussion

The table below summarizes the accuracy of each model on the training, validation, and testing sets:

|                          |Training|Validation|Testing  |       
|--------------------------|--------|----------|---------|
|Baseline SVM              |68.6%   |77.3%     |65.3%    |
|2 Layer ANN, 50 batch size|83.1%   |81.3%     |**90.7%**|
|2 Layer ANN, 75 batch size|83.1%   |78.7%     |**84.0%**|
|3 Layer ANN               |85.4%   |82.7%     |**86.7%**|
|CNN                       |66.3%   |56.0%     |60.0%    |


